In [ ]:
# Imports
import pandas as pd
import sqlite3

In [ ]:
# Get Data Frame
df = pd.read_csv('../data/raw/AB_NYC_2019.csv')
df.head()

In [ ]:
# Get each column type and check if there are more than one type per column to unify them
column_types = df.dtypes

formatted_types = {
    column: column_types[column].name if column_types[column] != 'bool' else 'boolean' 
    for column in df.columns
}

def get_types_per_column(df_columns):
    for column in df_columns:
        column_types = df[column].apply(type).unique()
        if len(column_types) > 1:
            type_names = [t.__name__ for t in column_types]
            print(f'{column}: <{type_names}>')
        else:
            print(f'{column}: <{column_types[0].__name__}>')

get_types_per_column(df.columns)

In [ ]:
# Clean data types
df['name'] = df['name'].astype(str)
df['host_name'] = df['host_name'].astype(str)
df['last_review'] = pd.to_datetime(df['last_review'], errors='coerce')

mean_date = df['last_review'].dropna().mean()
df['last_review'] = df['last_review'].fillna(mean_date)
df['last_review'] = df['last_review'].dt.date

mean_reviews_per_month = df['reviews_per_month'].dropna().mean()
df['reviews_per_month'] = df['reviews_per_month'].fillna(mean_reviews_per_month)

get_types_per_column(df.columns)